In [1]:
import numpy as np
import math

In [2]:
# g1(x1,x2)
def g1(x):
  return (2*x[1]) - x[0]

# g2(x1,x2)
def g2(x):
  return (2*x[0]) - x[1]

# g3(x1,x2)
def g3(x):
  return 1 - (x[0] + x[1])

# Ψ(x) = - Σ (log(gi(x)))    for i = 1, 2, ..., n

# For the given set of inequalities,
# Ψ(x) = - Σ (log(gi(x)))    for i = 1, 2, 3
# Ψ(x) = - (log(g1(x)) + log(g2(x)) + log(g3(x)))
# Ψ(x) = log(1 / (g1(x) * g2(x) * g3(x)))

def psi(x):
  return math.log(1.0 / (g1(x) * g2(x) * g3(x)))

# Minimising Ψ(x) = log(1 / (g1(x) * g2(x) * g3(x))) is equivalent
# to minimising the function f(x) = 1 / (g1(x) * g2(x) * g3(x))
# since log is a monotonically increasing function

def f(x):
  return (1.0 / (g1(x) * g2(x) * g3(x)))

# Filler function for denominator of gradient
def den(x):
  return (((g1(x))**2) * ((g2(x))**2) * ((g3(x))**2))

def grad(x):
  
  # df(x,y)/dx
  dx = -1 * (6 * (x[0]**2) - 6*x[0]*x[1] - 4*x[0] - 3*(x[1]**2) + 5*x[1]) / (den(x))

  #df(x,y)/dy
  dy = (3*(x[0]**2) + 6*x[0]*x[1] - 5*x[0] - 6*(x[1]**2) + 4*x[1]) / (den(x))
  
  return np.array([dx,dy])

In [3]:
# Backtracking Line Search Subroutine with alpha = 0.1, beta = 0.3

def lineSearch(v,d):
  di = d[0]
  dj = d[1]
  a = 0.1
  b = 0.3

  # maintain previously obtained value of t since x is set as x + t * d
  # and if x + t * d translates to a value < 0, it violates the domain of
  # log(x); in order to keep track of last obtained t for which x + t * d > 0,
  # the previous value is stored in a temporary variable
  t = 1.0
  prev = t

  # inequality translated to: f(x+td) - f(x) > alpha * t * (grad_f(x).d)

  while((f(v+t*d) > 0) and (f(v+t*d) - f(v) > a * t * np.dot(grad(v),d))):
    prev = t
    t = b*t

  # checks if x + t * d is in the domain of f(x)
  if(f(v+t*d) <= 0):
    return prev
  else:
    return t


# Generate Hessian of f(x,y) by approximating against (x1 + dx1,x2 + dx2) and
# (x1 - dx1, x2 - dx2) where dx1 and dx2 are defaulted to 1e-6
def hessian(x,dx = 1e-6):

  # Calculating d^2f(x)/dx1^2 and d^2f(x)/dx1x2
  hess = np.zeros((2,2))
  xp = x.copy()
  xn = x.copy()
  xp[0] += dx
  xn[0] -= dx
  hess[0,:] = (np.array(grad(xp)) - np.array(grad(xn)))/(2*dx)

  # Calculating d^2f(x)/dx1x2 and d^2f(x)/dx2^2
  yp = x.copy()
  yn = x.copy()
  yp[1] += dx
  yn[1] -= dx
  hess[1,:] = (np.array(grad(yp)) - np.array(grad(yn)))/(2*dx)

  return hess

In [4]:
# Combination Descent Algorithm with tolerance level n = 0.1

def combnDescent(x):
  n = 1e-5
  while(np.linalg.norm(grad(x))>n):
    
    # Get Hessian H(f(x,y))
    hes = hessian(x)

    # Check if hessian is positive definite; if yes, descent
    # direction is d = H^(-1)(f(x,y)).(-grad(f(x,y)))
    eigvals = np.linalg.eigvals(hes)
    if np.all(eigvals > 0):
      d = np.linalg.inv(hes).dot(-grad(x))

    # If not positive definite, descent direction is d = -grad(f(x,y))
    else:
      d = -grad(x)

    # Find t using Line Search Subroutine
    t = lineSearch(x,d)
    x = x + t*d
  
  # Return unconstrained minimum of f(x)
  return x

In [5]:
# Choose initial point as (0.25,0.25)
a = np.array([0.25,0.25])

print("[x,y] where Ψ(x) takes minimum value: ",end = "")
minarr = combnDescent(a)
print(minarr,end = "\n\n")

# Ψ(x) at unconstrained minimum
print("Minimum value of Ψ(x) as approximated by Combination Descent Algorithm: ",end = "")
print(psi(minarr))

[x,y] where Ψ(x) takes minimum value: [0.33333333 0.33333333]

Minimum value of Ψ(x) as approximated by Combination Descent Algorithm: 3.295836866004329
